In [1]:
## program for the search function 
import numpy as np
import pandas as pd
from os.path import join as oj
import os
import pandas as pd
import sys
import inspect
from datetime import datetime, timedelta
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)
sys.path.append(parentdir + '/modeling')
import load_data
from fit_and_predict import add_preds
from functions import merge_data
from viz import  viz_interactive, viz_static, viz_map_utils
import plotly.figure_factory as ff
import plotly
import re


#Load data and add prediction
def add_pre(df, var, name):
    h = df_county[var + '1-day'].copy()
    for i in range(2,8):
        h = np.vstack((h,df_county[var + str(i) +'-day']))
    df_county[name] = [h.T[i] for i in range(len(h.T))]


# generate html for individual counties
def generate_all_counties():
    print('generating html for counties')
    df_tab = df_county[['CountyName', 'StateName', 
             'deaths', 'cases','countyFIPS','pred_cases','pred_deaths','Predicted Deaths Intervals','Predicted Cases Intervals']]
    df_tab = df_tab.rename(columns={'CountyName': 'County', 'StateName': 'State',
                                'Predicted Deaths Intervals': 'pred_deaths_interval',
                                'Predicted Cases Intervals': 'pred_cases_interval'})
    dates = viz_map_utils.date_in_data(df_county)
    viz_interactive.viz_curves_all_counties(df_tab, oj(parentdir, 'results/All_counties/'), dates)
    print('succesfully generated all county html')

# Generate map plot 
def generate_map():
    print('generating search map')
    fips = df_county['countyFIPS']
    values = df_county['tot_cases']
    binning = []
    level = 6
    for i in range(level):
        binning.append(np.rint(2**(np.log2(max(values))/(level+1)*(i+1))))
    fig = ff.create_choropleth(fips=fips, values=values,
                           colorscale = ['#F7E8E4','#F5C8BB','#B96D67','#A83C3B',
                                '#8B2222','#5B0D0D','#5A2318'],binning_endpoints=binning,
                           legend_title='Cases'
                          )

    fig.update_layout(
        paper_bgcolor='rgba(0,0,0,255)',
        plot_bgcolor='rgba(0,0,0,255)',
        template='plotly_dark'
    )
    fig['layout'].update(width=1000, height=500, autosize=True)

    fig.write_image("results/search_map.png",width=900, height=450)
    
    print('succesfully generated search map')   
    
    return plotly.offline.plot(fig,
                include_plotlyjs='https://cdn.plot.ly/plotly-1.42.3.min.js',
                   output_type='div')


# update the html file (conbine search1 + search2 + search3)
def update_html(search2):
    id =re.search('<div id="([^ ]*)"',search2)
    print(id,id.group(1),id.group(0))
    with open('results/search1.html', 'r') as content_file:
        search1 = content_file.read()
    with open('results/search3.html', 'r') as content_file:
        search3 = content_file.read()
    f = open('results/search.html',"w+")
    f.write(search1+search2+search3.replace("map id to replace",id.group(1)))
    print('succesfully updated search html')



Loading BokehJS ...

In [2]:
if __name__ == '__main__':
    print('loading data...')
    NUM_DAYS_LIST = [1, 2, 3, 4, 5, 6, 7]
    df_county = load_data.load_county_level(data_dir=oj(parentdir, 'data')).fillna(0)
    df_county = add_preds(df_county, NUM_DAYS_LIST=NUM_DAYS_LIST, cached_dir=oj(parentdir, 'data')) # adds keys like "Predicted Deaths 1-day"

    ## orgnize predicts as array
    add_pre(df_county,'Predicted Cases ','pred_cases')
    add_pre(df_county,'Predicted Deaths ','pred_deaths')

    #generate_all_counties()
    #search2 = generate_map()
    #update_html(search2)

loading data...
loading county-level data...
loaded and merged COVID-19 cases/deaths data successfully


In [15]:
df_tab = df_county[['CountyName', 'StateName', 
             'deaths', 'cases','countyFIPS','pred_cases','pred_deaths','Predicted Deaths Intervals','Predicted Cases Intervals']].head(1)
df_tab = df_tab.rename(columns={'CountyName': 'County', 'StateName': 'State',
                                'Predicted Deaths Intervals': 'pred_deaths_interval',
                                'Predicted Cases Intervals': 'pred_cases_interval'})
dates = viz_map_utils.date_in_data(df_county)
viz_interactive.viz_curves_all_counties(df_tab, parentdir, dates)

ValueError: Invalid property specified for object of type plotly.graph_objs.Layout: 'xaxis2_range'

    Valid properties:
        activeshape
            :class:`plotly.graph_objects.layout.Activeshape`
            instance or dict with compatible properties
        angularaxis
            :class:`plotly.graph_objects.layout.AngularAxis`
            instance or dict with compatible properties
        annotations
            A tuple of
            :class:`plotly.graph_objects.layout.Annotation`
            instances or dicts with compatible properties
        annotationdefaults
            When used in a template (as
            layout.template.layout.annotationdefaults), sets the
            default property values to use for elements of
            layout.annotations
        autosize
            Determines whether or not a layout width or height that
            has been left undefined by the user is initialized on
            each relayout. Note that, regardless of this attribute,
  

In [41]:
print(parentdir)

/usr/local/google/home/danqingwang/covid19-severity-prediction


In [3]:
from bokeh.sampledata import us_states, us_counties
from bokeh.plotting import figure, show, output_notebook, output_file, save
from bokeh import palettes
from bokeh.models import ColorBar,HoverTool,LinearColorMapper,ColumnDataSource,FixedTicker, LogColorMapper
output_notebook()
import re
import numpy as np
from modeling import fit_and_predict
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import plot
from plotly.subplots import make_subplots
import json
import plotly.express as px
import plotly
import pandas as pd
from datetime import datetime, timedelta

row = df_tab.iloc[0]
state,county,FIPS = row['State'],row['County']+" County",row['countyFIPS']
newdates = [dates[-1] + timedelta(days =i) for i in range(0,8)]
fig = make_subplots(specs=[[{"secondary_y": True}]])
traces = []
keys_curves=['deaths','cases']
credstr ='rgb(234, 51, 86)'
cbluestr = 'rgb(57, 138, 242)'
fill_strings = ['rgba(234, 51, 86,0.1)','rgba(57, 138, 242,0.1)']
color_strings = [credstr, cbluestr]




x = [d.strftime("%m/%d") for d in dates]
for j, key_curve in enumerate(keys_curves):
        curve = row[key_curve]
        traces = []
        traces.append(go.Scatter(x=dates,
                    y=curve,
                    showlegend=True,
                    visible=True, #key == key0,# False, #key == key0,
                    name=key_curve,
                    line=dict(color=color_strings[j], width=4))        )
        low = np.hstack((curve[-1],[x[0] for x in row['pred_'+key_curve+'_interval']]))
        up = np.hstack((curve[-1],[x[1] for x in row['pred_'+key_curve+'_interval']]))
        traces.append(go.Scatter(
                    name = 'Upper Bound',
                    showlegend= False,
                    x = newdates,
                    y = up,
                    line=dict(dash ='solid',color=fill_strings[j], width=4)))
        traces.append(go.Scatter(x= newdates,
                    y=np.hstack(([curve[-1]],row['pred_'+key_curve])),
                    showlegend= False,
                    visible=True,
                    mode = 'lines',
                    name = 'prediction on '+key_curve,
                    line=dict(dash ='dash',color=color_strings[j], width=4),
                    fillcolor = fill_strings[j],
                    fill ='tonexty'))
        traces.append(go.Scatter(
                    name = 'Lower Bound',
                    visible = True,
                    showlegend= False,
                    x = newdates,
                    y = low,
                    fill ='tonexty',
                    fillcolor = fill_strings[j],
                    line=dict(dash ='solid',color=fill_strings[j], width=4)))
        for trace in traces:
            fig.add_trace(trace, secondary_y=key_curve=="deaths")
        
        # initialize xaxis2 and yaxis2
fig['layout']['xaxis'] = {}
fig['layout']['yaxis'] = {}
        
        
        # Edit layout for subplots
fig.layout.xaxis.update({'domain': [0.05, .95]})
fig.layout.xaxis.update({'title': 'Date'})

y1 = max([x[1] for x in row['pred_cases_interval']])
y2 = max([x[1] for x in row['pred_deaths_interval']])

fig.layout.yaxis.update({'domain': [0, .95]})
fig.layout.yaxis.update({'title': 'Cases','rangemode':'tozero','dtick': y1/5},range=[0,y1])
fig.update_yaxes(title_text="Deaths",rangemode = 'tozero',dtick = y2*1.3/5, range=[0,y2*1.3], secondary_y=True)


        # Update the margins to add a title and see graph x-labels.
fig.layout.margin.update({'t':150, 'b':120})
        
fig.layout.update(
            title ={'text':state+' '+county+ '<br>' + 'FIPS:'+row['countyFIPS'], 'y':0.93,
        'x':0.05, 
        'yanchor': 'top'},
        font=dict(
        size=19))


fig.update_layout(xaxis_range=[datetime(2020, 3, 25),
                               newdates[-1]])
fig.layout.template = 'plotly_dark'
plotly.offline.plot(fig, filename='test.html', auto_open=True)


Loading BokehJS ...

NameError: name 'df_tab' is not defined